In [1]:
# only for testing
# use main.py for training


# things to improve performance (speed)
# use torch.utils.random_split https://www.programcreek.com/python/example/125046/torch.utils.data.random_split
# https://pytorch.org/tutorials/recipes/recipes/tuning_guide.html?highlight=checkpoint
# implement "channels Last Memory Format" (beta) https://pytorch.org/tutorials/intermediate/memory_format_tutorial.html

In [1]:
# sample image
import torch

from dataset.utils import pNormalize
from dataset.datasets import sentinel
from model.models import UNET

from torchvision import transforms
from torch.utils.data import DataLoader
q_hi = torch.tensor([2102.0, 1716.0, 1398.0, 4732.0, 2434.42919921875, 3701.759765625, 4519.2177734375, 4857.7734375, 3799.80322265625, 3008.8935546875])
q_lo = torch.tensor([102.0, 159.0, 107.0, 77.0, 106.98081970214844, 79.00384521484375, 86.18966674804688, 70.40167236328125, 50.571197509765625, 36.95356750488281])
        
norm = pNormalize(
                maxPer=q_hi,
                minPer=q_lo
)


# Create experimental dataset and loader, rgb=True for 3 channels (default = False)
test_set = sentinel(root_dir='processed-data/dsen_2_256_new_split', img_transform=norm)
test_loader = DataLoader(test_set, batch_size=10, num_workers=0)

print(len(test_set))

# access one image from dataset
img, label = test_set[0]
#print('image shape:', img.shape)


# or  batch from DataLoader
dataiter = iter(test_loader)
images, labels = dataiter.next()





# define model
model = UNET(in_channels=images.shape[1])
model.load_state_dict(torch.load('to/model_epoch_95.pt',map_location=torch.device('cpu')))
# disable gradients etc. model.train() for training https://medium.com/jun94-devpblog/pytorch-6-model-train-vs-model-eval-no-grad-hyperparameter-tuning-3812c216a3bd
model.eval()

output = model(images)

#pred = torch.argmax(output, dim=1)
     # prediction
pred= torch.nn.functional.softmax(output,dim=1)
pred = torch.argmax(pred,dim=1)


#print('prediction:', pred.shape)
#print('labels:', labels.shape)

print('output:',output.shape)
print('labl', labels.shape)
print('pred:',pred.shape)

2524
output: torch.Size([10, 28, 256, 256])
labl torch.Size([10, 256, 256])
pred: torch.Size([10, 256, 256])


In [2]:
from train.metrics import plot_sample, plot_batch

for idx,(images,labels) in enumerate(test_loader):
#dataiter = iter(test_loader)
#images, labels = dataiter.next()
    with torch.no_grad():
        output = model(images)
        pred= torch.nn.functional.softmax(output,dim=1)
        pred = torch.argmax(pred,dim=1)
        plot_batch(pred,labels, images)
        
        for i in range(pred.shape[0]):
            plot_sample(pred[i,:,:],labels[i,:,:], images[i,0:3,:,:]  )
            break
            #plot_pred_lbl_rgb(pred[i,:,:],labels[i,:,:], images[i,0:3,:,:]  )
        if idx==0:
            break

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping i

In [6]:
import torch
from preprocess.classDict import class_dict
counts= torch.tensor([  
    535419,   138327,   619285,  1589119,  4156781,  4242609, 1600499, 4293367,
    644562,  9073507,   614877,    70747,   419341,  742613,  159342,   185612,
    2195461,  2186212, 25360177,     3413, 21960385, 0,       0,      71047030,
    1430349,    12117,  1556533, 10575180],
    dtype=torch.int32)

ce_weights =torch.tensor([
    0.0000e+00, 5.1362e+02, 1.1472e+02, 4.4708e+01, 1.7092e+01, 1.6746e+01,
    4.4391e+01, 1.6548e+01, 1.1023e+02, 7.8302e+00, 1.1555e+02, 1.0042e+03,
    1.6943e+02, 9.5672e+01, 4.4588e+02, 3.8277e+02, 3.2361e+01, 3.2498e+01,
    2.8015e+00, 2.0817e+04, 3.2352e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
    4.9671e+01, 5.8634e+03, 4.5644e+01, 6.7183e+00])

n_samples = torch.sum(counts[1:28])
n_classes = counts.shape[0]-3

weights  = n_samples/(n_classes*counts)
weights[torch.where(counts==0)] = 0
#torch.max(weights)
#torch.min(weights[torch.where(weights!=0)])
#torch.max(ce_weights)
#torch.min(ce_weights[torch.where(ce_weights!=0)])
weights[9] += 100
ce_weights[9] +=100
weights[18]+=10
weights[20] +=10
weights[23] +=10
weights[27] +=1

for ii, v in enumerate(class_dict.values()):
    i=ii+1
    print(i,round(ce_weights[i].item(),2),' ',round(weights[i].item(),2),' ',v['class_name'],'\n')

weights[9]

ce_weights

1 513.62   47.68   Continuous Urban Fabric (S.L. &amp;gt; 80%) 

2 114.72   10.65   Discontinuous Dense Urban Fabric (S.L. : 50% - 80%) 

3 44.71   4.15   Discontinuous Medium Density Urban Fabric (S.L. : 30% - 50%) 

4 17.09   1.59   Discontinuous Low Density Urban Fabric (S.L. : 10% - 30%) 

5 16.75   1.55   Discontinuous Very Low Density Urban Fabric (S.L. &amp;lt; 10%) 

6 44.39   4.12   Isolated Structures 

7 16.55   1.54   Industrial, commercial, public, military and private units 

8 110.23   10.23   Fast transit roads and associated land 

9 107.83   100.73   Other roads and associated land 

10 115.55   10.73   Railways and associated land 

11 1004.2   93.22   Port areas 

12 169.43   15.73   Airports 

13 95.67   8.88   Mineral extraction and dump sites 

14 445.88   41.39   Construction sites 

15 382.77   35.53   Land without current use 

16 32.36   3.0   Green urban areas 

17 32.5   3.02   Sports and leisure facilities 

18 2.8   10.26   Arable land (annual crops) 

19

tensor([0.0000e+00, 5.1362e+02, 1.1472e+02, 4.4708e+01, 1.7092e+01, 1.6746e+01,
        4.4391e+01, 1.6548e+01, 1.1023e+02, 1.0783e+02, 1.1555e+02, 1.0042e+03,
        1.6943e+02, 9.5672e+01, 4.4588e+02, 3.8277e+02, 3.2361e+01, 3.2498e+01,
        2.8015e+00, 2.0817e+04, 3.2352e+00, 0.0000e+00, 0.0000e+00, 1.0000e+00,
        4.9671e+01, 5.8634e+03, 4.5644e+01, 6.7183e+00])

In [ ]:
# run this to make sure tensorboard is working
#!conda list | grep tensorboard
import torch
#!conda install tensorboard -y
from torch.utils.tensorboard import SummaryWriter
print(torch.__version__)

#make sure this executes
SummaryWriter()

1.12.0.dev20220425


In [4]:
#visualise images in tensorboard

dataiter = iter(test_loader)
images, labels = dataiter.next()

# Create a grid from the images and show them
# https://pytorch.org/vision/main/auto_examples/plot_visualization_utils.html#sphx-glr-auto-examples-plot-visualization-utils-py
img_grid = torchvision.utils.make_grid(images)
#show(img_grid)
#matplotlib_imshow(img_grid, one_channel=False)
#print('  '.join(classes[labels[j]] for j in range(4))
# Default log_dir argument is "runs" - but it's good to be specific
# torch.utils.tensorboard.SummaryWriter is imported above
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/visualise_data')

# Write image data to TensorBoard log dir
writer.add_image('Sat Images', img_grid)
writer.flush()

# To view, start TensorBoard on the command line with:
#  tensorboard --logdir=runs
# ...and open a browser tab to http://localhost:6006/

#if on remote
## on remote
##tensorboard --logdir <path> --port 6006
## forward everything on port 6006 on server to 
##ssh -L 16006:127.0.0.1:6006 -i vmexjobb_key.pem exjobbare@vdexjobb.westeurope.cloudapp.azure.com

NameError: name 'training_loader' is not defined

In [ ]:
#visualizing model in tensorboard


# Again, grab a single mini-batch of images
dataiter = iter(test_loader)
images, labels = dataiter.next()

# add_graph() will trace the sample input through your model,
# and render it as a graph.
writer.add_graph(model, images)
writer.flush()

/media/oskar/ESSD/deepSat/model/models.py:56: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if x.shape != concat_layer.shape:
